In [1]:
import nemo.collections.asr as nemo_asr
import torch
import numpy as np
import pytorch_lightning as ptl
from omegaconf import OmegaConf
import os

def load_config(config_path):
    config = OmegaConf.load(config_path)
    config = OmegaConf.to_container(config, resolve=True)
    config = OmegaConf.create(config)
    return config

# Conformer
# model_path = "/home/nhan/NovaIntechs/models/conformer_small/Conformer_small_Model_Language_vi--val_wer=0.0426-epoch=98.ckpt"
# config_path = "/home/khoatlv/ASR-NEMO/config/conformer_small_ctc_bpe.yaml"
# config = load_config(config_path)
asr_model = nemo_asr.models.EncDecCTCModelBPE.restore_from("/home/khoatlv/ASR-NEMO/models/conformer/Conformer_small_epoch=98.nemo")
lm_path = "/home/khoatlv/ASR-NEMO/n_gram_lm/n_gram_lm_model/6-conformer-small-gram_trained.bin"

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



[NeMo I 2022-04-03 11:51:31 mixins:146] Tokenizer SentencePieceTokenizer initialized with 256 tokens


[NeMo W 2022-04-03 11:51:32 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /home/nhan/NovaIntechs/data/ASR_Data/manifests/train_manifest_processed.json
    sample_rate: 16000
    max_duration: 16.7
    min_duration: 0.1
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    shuffle: true
    batch_size: 32
    pin_memory: true
    trim_silence: true
    use_start_end_token: true
    normalize_transcripts: false
    num_workers: 16
    
[NeMo W 2022-04-03 11:51:32 modelPT:155] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manif

[NeMo I 2022-04-03 11:51:32 features:255] PADDING: 0
[NeMo I 2022-04-03 11:51:32 features:272] STFT using torch
[NeMo I 2022-04-03 11:51:37 save_restore_connector:157] Model EncDecCTCModelBPE was successfully restored from /home/khoatlv/ASR-NEMO/models/conformer/Conformer_small_epoch=98.nemo.


In [25]:
from nemo.core.classes import IterableDataset
from nemo.core.neural_types import NeuralType, AudioSignal, LengthsType
import torch
import nemo
from torch.utils.data import DataLoader

TOKEN_OFFSET = 100
# simple data layer to pass audio signal
class AudioDataLayer(IterableDataset):
    @property
    def output_types(self):
        return {
            'audio_signal': NeuralType(('B', 'T'), AudioSignal(freq=self._sample_rate)),
            'a_sig_length': NeuralType(tuple('B'), LengthsType()),
        }

    def __init__(self, sample_rate):
        super().__init__()
        self._sample_rate = sample_rate
        self.output = True
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if not self.output:
            raise StopIteration
        self.output = False
        return torch.as_tensor(self.signal, dtype=torch.float32), \
               torch.as_tensor(self.signal_shape, dtype=torch.int64)
        
    def set_signal(self, signal):
        self.signal = signal.astype(np.float32)/32768.
        self.signal_shape = self.signal.size
        self.output = True

    def __len__(self):
        return 1

data_layer = AudioDataLayer(sample_rate=16000)
data_loader = DataLoader(data_layer, batch_size=1, collate_fn=data_layer.collate_fn)

vocab = asr_model.decoder.vocabulary
vocab = [chr(idx + TOKEN_OFFSET) for idx in range(len(vocab))]
ids_to_text_func = asr_model.tokenizer.ids_to_text

beam_search_lm = nemo_asr.modules.BeamSearchDecoderWithLM(
    vocab=list(vocab),
    beam_width=200,
    alpha=2, beta=1.5,
    lm_path=lm_path,
    num_cpus=max(os.cpu_count(), 1),
    input_tensor=False
)

In [40]:
audio_name = "FPTOpenSpeechData_Set002_V0.1_011692.wav"
AUDIO_FILENAME = os.path.join("/home/khoatlv/data/FPT/wav", audio_name)
# signal, sr = librosa.load(AUDIO_FILENAME, sr=16000)

# softmax implementation in NumPy
def softmax(logits):
    e = np.exp(logits - np.max(logits))
    return e / e.sum(axis=-1).reshape([logits.shape[0], 1])

# let's do inference once again but without decoder
with torch.no_grad():
    logits = asr_model.transcribe([AUDIO_FILENAME], batch_size=1, logprobs=True)[0]
probs = softmax(logits)

preds = np.argmax(probs, axis=1)
preds_tensor = torch.tensor(preds, device='cuda').unsqueeze(0)
pred_text = asr_model._wer.ctc_decoder_predictions_tensor(preds_tensor)[0]
print(f"Greedy decoding: {pred_text}")

with nemo.core.typecheck.disable_checks():
    beams_seach_result = beam_search_lm.forward(log_probs=[probs], log_probs_length=None,)[0]
    # print(beams_seach_result)
    
# for beams_idx, beams in enumerate(beams_seach_result):
for candidate_idx, candidate in enumerate(beams_seach_result):
    pred_text = ids_to_text_func([ord(c) - TOKEN_OFFSET for c in candidate[1]])
    print(pred_text)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Greedy decoding: ai cũng khen cô con dâu nhanh nhạng tươi tắn nói năng linh học
ai cũng khen cô con dâu nhanh nhẹn tươi tắn nói năng linh
ai cũng khen cô con dâu nhanh nhẹn tươi tắn nói năng linh học
ai cũng khen cô con dâu nhanh nhẹn tươi tắn có năng linh
ai cũng khen cô con dâu nhanh nhẹn tươi tắn nói năng lên
ai cũng khen cô con dâu nhanh nhẹn tươi tắn nói năng lên học
ai cũng khen cô con dâu nhanh nhẹn tươi tắn nó năng linh
ai cũng khen cô con dâu nhanh nhẹn tươi tắn nói năng linh họ
ai cũng khen cô con dâu nhanh nhẹn tươi tắn có năng linh học
ai cũng khen cô con dâu nhanh nhẹn tươi tắn nói năng linh h
ai cũng khen cô con dâu nhanh nhẹn tươi tắn nó năng linh học
ai cũng khen cô con dâu nhanh nhẹn tươi tắn nói năng linh ho
ai cũng khiến cô con dâu nhanh nhẹn tươi tắn nói năng linh
ai cũng khen cô con dâu nhanh nhẹn tươi tắn có năng lên
ai cũng khen cô con dâu nhanh nhẹn tươi tắn có năng lực
ai cũng ken cô con dâu nhanh nhẹn tươi tắn nói năng linh
ai cũng khen cô con dâu nhanh nhẹn t

In [24]:
it = tqdm(
    range(int(np.ceil(len(all_probs) / 1))),
    desc=f"Beam search decoding with width={200}, alpha={2}, beta={2.5}",
    ncols=120,
)

for batch_idx in it:
    # disabling type checking
    with nemo.core.typecheck.disable_checks():
        probs_batch = all_probs[batch_idx * 1 : (batch_idx + 1) * 1]
        beams_batch = beam_search_lm.forward(log_probs=probs_batch, log_probs_length=None,)

    with nemo.core.typecheck.disable_checks():
        beams_batch = beam_search_lm.forward(log_probs=[probs], log_probs_length=None,)
        # print(beams_batch)
    for beams_idx, beams in enumerate(beams_batch):
        for candidate_idx, candidate in enumerate(beams):
            pred_text = ids_to_text_func([ord(c) - TOKEN_OFFSET for c in candidate[1]])
            print(pred_text)

Beam search decoding with width=200, alpha=2, beta=2.5:   0%|                                     | 0/1 [00:00…

ai cũng khen cô con dâu nhanh nhẹn tươi tắn nói năng linh
ai cũng khen cô con dâu nhanh nhẹn tươi tắn nói năng linh học
ai cũng khen cô con dâu nhanh nhẹn tươi tắn có năng linh
ai cũng khen cô con dâu nhanh nhẹn tươi tắn nói năng lên
ai cũng khen cô con dâu nhanh nhẹn tươi tắn nói năng lên học
ai cũng khen cô con dâu nhanh nhẹn tươi tắn nó năng linh
ai cũng khen cô con dâu nhanh nhẹn tươi tắn nói năng linh họ
ai cũng khen cô con dâu nhanh nhẹn tươi tắn có năng linh học
ai cũng khen cô con dâu nhanh nhẹn tươi tắn nói năng linh h
ai cũng khen cô con dâu nhanh nhẹn tươi tắn nó năng linh học
ai cũng khen cô con dâu nhanh nhẹn tươi tắn nói năng linh ho
ai cũng khiến cô con dâu nhanh nhẹn tươi tắn nói năng linh
ai cũng khen cô con dâu nhanh nhẹn tươi tắn có năng lên
ai cũng khen cô con dâu nhanh nhẹn tươi tắn có năng lực
ai cũng ken cô con dâu nhanh nhẹn tươi tắn nói năng linh
ai cũng khen cô con dâu nhanh nhẹn tươi tắn có năng lên học
ai cũng khen cô con dâu nhanh nhẹn tươi tắn nó năng lên
